In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import h5py
%matplotlib inline


/Users/diegobarbosa/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
file1=h5py.File('particle_positions.h5')
dset=file1['positions']

In [4]:
dset.shape

(134217728, 3)

Tenemos 134217728 partículas con posiciones en x,y,z en una caja de longitud L=2000MPc/h y queremos 
hacer asignación de masas usando el algorítmo CIC

Vamos a tomar las posiciones de x,y,z 

In [8]:
xpos=dset[:,0];
ypos=dset[:,1];
zpos=dset[:,2];

In [11]:
Lx=2000.0
Ly=2000.0
Lz=2000.0
N=256

In [12]:
Deltax=(Lx/N)
Deltay=(Ly/N)
Deltaz=(Lz/N)

Para calcular la densidad, primero necesitamos un arreglo de tamaño  $N_{gx}\times N_{gy}\times N_{gz}$, para nuestro caso es $N^3$. Definamos esa variable:

In [13]:
den3D=np.zeros((N,N,N))

La función forma de la partícula en este caso es un paralelepipedo rectangular  de volume, $\Delta x \Delta y \Delta z$. En este esquema se involucran las ocho celdas más cercanas para cada partícula. Este esquema es conocido como Cloud-in-Cell (CiC).

Consideremos, el esquema de asignación CiC para una partícula con coordenadas $(x_p,y_p,z_p)$. La celda que contiene 
la partícula tiene índices dados por: $i=floor(x_p/\Delta x)$, $j=floor(y_p/\Delta y)$, $k=floor(z_p/\Delta z)$

Definamos una función que nos retorne estos valores:

In [16]:
def inx(xp,Dx):
    i=int(math.floor(xp/Dx))
    return i

In [17]:
def iny(yp,Dy):
    j=int(math.floor(yp/Dy))
    return j

In [18]:
def inz(zp,Dz):
    k=int(math.floor(zp/Dz))
    return k

Debemos considerar el centro de la celda el cuál va estar definido por $(x_c,y_c,z_c)=(i\Delta x, j \Delta y, k\Delta z)$

In [20]:
def CCx(xp,Dx):
    xc=inx(xp,Dx)*Dx
    return xc

In [21]:
def CCy(yp,Dy):
    yc=iny(yp,Dy)*Dy
    return yc

In [22]:
def CCz(zp,Dz):
    zc=iny(zp,Dz)*Dz
    return zc

La densisdad contribuye a la celda principal y a siete celdas vecinas indexadas por $(i,j,k), (ii,j,k),(i,jj,k),(i,j,kk),(ii,jj,k),(ii,j,kk),(i,jj,kk)$ y $(ii,jj,kk)$

Donde los índices $ii$,$jj$ y $kk$ están definidos por: $ii=mod(i+1, N_{gx})$,$jj=mod(j+1, N_{gy})$ y 
$kk=mod(k+1, N_{gz})$

In [24]:
def inxx(xp,Dx,Nx):
    ii=int((inx(xp,Dx)+1)%Nx)
    return ii

In [25]:
def inyy(yp,Dy,Ny):
    jj=int((iny(yp,Dy)+1)%Ny)
    return jj

In [26]:
def inzz(zp,Dz,Nz):
    kk=int((inz(zp,Dz)+1)%Nz)
    return kk

Definamos ahora las cantidades $dx=\frac{x_p}{\Delta x}-i$, $dy=\frac{y_p}{\Delta y}-j$, $dz=\frac{z_p}{\Delta z}-k$

y $t_x=1-dx$, $t_y=1-dy$, $t_z=1-dz$

In [27]:
def delx(xp,Dx):
    dx=(xp/Dx)-inx(xp,Dx)
    return dx

In [28]:
def dely(yp,Dy):
    dy=(yp/Dy)-iny(yp,Dy)
    return dy

In [29]:
def delz(zp,Dz):
    dz=(zp/Dz)-inz(zp,Dz)
    return dz

In [30]:
def ti(xp,Dx):
    tx=1-delx(xp,Dx)
    return tx

In [31]:
def tj(yp,Dy):
    ty=1-dely(yp,Dy)
    return ty

In [32]:
def tk(zp,Dz):
    tz=1-delz(zp,Dz)
    return tz

Las contribuciones a las ocho celdas están dadas por $W(\vec{x_P}-x_{i,j,k})=t_x t_y t_z$

$W(\vec{x_P}-x_{ii,j,k})=d_x t_y t_z$

$W(\vec{x_P}-x_{i,jj,k})=t_x d_y t_z$

$W(\vec{x_P}-x_{i,j,kk})=t_x t_y d_z$

$W(\vec{x_P}-x_{ii,jj,k})=d_x d_y t_z$

$W(\vec{x_P}-x_{ii,j,kk})=d_x t_y d_z$

$W(\vec{x_P}-x_{i,jj,kk})=t_x d_y d_z$

$W(\vec{x_P}-x_{ii,jj,kk})=d_x d_y d_z$

Sumando sobre todas las partículas resulta en el calculo de $\rho$ en la grid. Pregunta: Sabemos que $\rho(x_c)=Cons\sum_{i}^{N_p}W(\vec{x_i}-\vec{x_{ijk}})$

In [35]:
def rho3D(xp,yp,zp,Dx,Dy,Dz,Nx,Ny,Nz,den3D):
    i=int(inx(xp,Dx))
    j=int(iny(yp,Dy))
    k=int(inz(yp,Dz))
    ii=inxx(xp,Dx,Nx)
    jj=inyy(yp,Dy,Ny)
    kk=inyy(zp,Dz,Nz)
    den3D[i,j,k]=den3D[i,j,k]+(ti(xp,Dx)*tj(yp,Dy)*tk(zp,Dz))
    den3D[ii,j,k]=den3D[ii,j,k]+(delx(xp,Dx)*tj(yp,Dy)*tk(zp,Dz))
    den3D[i,jj,k]=den3D[i,jj,k]+(ti(xp,Dx)*dely(yp,Dy)*tk(zp,Dz))
    den3D[i,j,kk]=den3D[i,j,kk]+(ti(xp,Dx)*tj(yp,Dy)*delz(zp,Dz))
    den3D[ii,jj,k]=den3D[ii,jj,k]+(delx(xp,Dx)*dely(yp,Dy)*tk(zp,Dz))
    den3D[ii,j,kk]=den3D[ii,j,kk]+(delx(xp,Dx)*tj(yp,Dy)*delz(zp,Dz))
    den3D[i,jj,kk]=den3D[i,jj,kk]+(ti(xp,Dx)*dely(yp,Dy)*delz(zp,Dz))
    den3D[ii,jj,kk]=den3D[ii,jj,kk]+(delx(xp,Dx)*dely(yp,Dy)*delz(zp,Dz))

In [36]:
def suma(x,y,z,Dx,Dy,Dz,Nx,Ny,Nz,Np):
    den3d=np.zeros((Nx,Ny,Nz))
    for i in range(0,Np):
        xp=x[i]
        yp=y[i]
        zp=z[i]
        rho3D(xp,yp,zp,Dx,Dy,Dz,Nx,Ny,Nz,den3d)
    return den3d

In [38]:
Den3D=suma(xpos,ypos,zpos,Deltax,Deltay,Deltaz,256,256,256,len(xpos))

In [39]:
f=h5py.File('density2.h5','w')

In [40]:
dset=f.create_dataset('density2',data=Den3D)

In [41]:
f.close()